## Imports

In [1]:
import datetime
import os

In [2]:
import sklearn_evaluation as skeval
import xgboost as xgb

In [3]:
from scipy.stats import randint, uniform
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import *
from xgboost import XGBClassifier

## Config

In [4]:
RANDOM_SEED = 42

In [5]:
data_folder = os.path.abspath(os.path.join(os.curdir, os.pardir, 'data')) + os.path.sep
aux_data_folder = os.path.join(data_folder, 'aux') + os.path.sep
preproc_data_folder = os.path.join(data_folder, 'preproc') + os.path.sep
features_data_folder = os.path.join(data_folder, 'features') + os.path.sep
submissions_data_folder = os.path.join(data_folder, 'submissions') + os.path.sep

## Read Data

In [6]:
X = load(features_data_folder + 'X_train_summary_stats.pickle')

In [7]:
y = load(features_data_folder + 'y_train.pickle')

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.15,
    random_state=RANDOM_SEED,
    stratify=y
)

In [9]:
print('X train:', X_train.shape)
print('y train:', y_train.shape)
print('X val:  ', X_val.shape)
print('y val:  ', y_val.shape)

X train: (343646, 12)
y train: (343646,)
X val:   (60644, 12)
y val:   (60644,)


## Train Model

In [10]:
positive_imbalance_ratio = np.count_nonzero(y_train == 0) / np.count_nonzero(y_train == 1)

In [11]:
print('Positive imbalance ratio:', positive_imbalance_ratio)

Positive imbalance ratio: 1.7085825983463778


In [12]:
xgb_params = {
    'objective': 'binary:logistic',
    'n_estimators': 100,
    'seed': RANDOM_SEED,
    'learning_rate': 0.15,
    'max_depth': 6,
    'subsample': 0.75,
    'colsample_bytree': 0.4,
    'scale_pos_weight': positive_imbalance_ratio,
    'silent': 1,
}

### Search for the best parameters

In [13]:
random_search_parameters = {
    'n_estimators': randint(200, 1500),
    'learning_rate': uniform(0.01, 0.1 - 0.01),
    'max_depth': randint(5, 16),
    'subsample': uniform(0.2, 1.0 - 0.2),
    'colsample_bytree': uniform(0.2, 1.0 - 0.2),
}

In [14]:
model = XGBClassifier(**xgb_params)

In [15]:
def log_loss_scorer(estimator, X, y):
    return -log_loss(y, estimator.predict_proba(X)[:, -1])

In [16]:
randomized_search = RandomizedSearchCV(
    model,
    random_search_parameters,
    n_jobs=1,
    n_iter=250,
    cv=3,
    scoring=log_loss_scorer,
    refit=True,
    verbose=2,
    random_state=RANDOM_SEED,
)

In [ ]:
%%time
model = randomized_search.fit(X_train, y_train)

Fitting 3 folds for each of 250 candidates, totalling 750 fits
[CV] colsample_bytree=0.7426732318, learning_rate=0.0330769850738, max_depth=9, n_estimators=778, subsample=0.290430557588 
[CV]  colsample_bytree=0.7426732318, learning_rate=0.0330769850738, max_depth=9, n_estimators=778, subsample=0.290430557588, total=  49.9s
[CV] colsample_bytree=0.7426732318, learning_rate=0.0330769850738, max_depth=9, n_estimators=778, subsample=0.290430557588 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   52.4s remaining:    0.0s


[CV]  colsample_bytree=0.7426732318, learning_rate=0.0330769850738, max_depth=9, n_estimators=778, subsample=0.290430557588, total=  49.0s
[CV] colsample_bytree=0.7426732318, learning_rate=0.0330769850738, max_depth=9, n_estimators=778, subsample=0.290430557588 
[CV]  colsample_bytree=0.7426732318, learning_rate=0.0330769850738, max_depth=9, n_estimators=778, subsample=0.290430557588, total=  48.5s
[CV] colsample_bytree=0.483088516261, learning_rate=0.0483813466299, max_depth=5, n_estimators=774, subsample=0.348129242398 
[CV]  colsample_bytree=0.483088516261, learning_rate=0.0483813466299, max_depth=5, n_estimators=774, subsample=0.348129242398, total=  27.3s
[CV] colsample_bytree=0.483088516261, learning_rate=0.0483813466299, max_depth=5, n_estimators=774, subsample=0.348129242398 
[CV]  colsample_bytree=0.483088516261, learning_rate=0.0483813466299, max_depth=5, n_estimators=774, subsample=0.348129242398, total=  26.5s
[CV] colsample_bytree=0.483088516261, learning_rate=0.0483813466

[CV]  colsample_bytree=0.823312697817, learning_rate=0.0792435557242, max_depth=14, n_estimators=440, subsample=0.306663570056, total=  49.6s
[CV] colsample_bytree=0.359774110236, learning_rate=0.065439990793, max_depth=11, n_estimators=1392, subsample=0.43874656664 
[CV]  colsample_bytree=0.359774110236, learning_rate=0.065439990793, max_depth=11, n_estimators=1392, subsample=0.43874656664, total= 1.8min
[CV] colsample_bytree=0.359774110236, learning_rate=0.065439990793, max_depth=11, n_estimators=1392, subsample=0.43874656664 
[CV]  colsample_bytree=0.359774110236, learning_rate=0.065439990793, max_depth=11, n_estimators=1392, subsample=0.43874656664, total= 1.7min
[CV] colsample_bytree=0.359774110236, learning_rate=0.065439990793, max_depth=11, n_estimators=1392, subsample=0.43874656664 
[CV]  colsample_bytree=0.359774110236, learning_rate=0.065439990793, max_depth=11, n_estimators=1392, subsample=0.43874656664, total= 1.7min
[CV] colsample_bytree=0.986150310568, learning_rate=0.081

[CV]  colsample_bytree=0.973316072118, learning_rate=0.0939355714511, max_depth=12, n_estimators=1403, subsample=0.855738826669, total= 3.6min
[CV] colsample_bytree=0.973316072118, learning_rate=0.0939355714511, max_depth=12, n_estimators=1403, subsample=0.855738826669 
[CV]  colsample_bytree=0.973316072118, learning_rate=0.0939355714511, max_depth=12, n_estimators=1403, subsample=0.855738826669, total= 3.6min
[CV] colsample_bytree=0.765327444916, learning_rate=0.0488237553815, max_depth=9, n_estimators=918, subsample=0.702899200562 
[CV]  colsample_bytree=0.765327444916, learning_rate=0.0488237553815, max_depth=9, n_estimators=918, subsample=0.702899200562, total= 1.5min
[CV] colsample_bytree=0.765327444916, learning_rate=0.0488237553815, max_depth=9, n_estimators=918, subsample=0.702899200562 
[CV]  colsample_bytree=0.765327444916, learning_rate=0.0488237553815, max_depth=9, n_estimators=918, subsample=0.702899200562, total= 1.5min
[CV] colsample_bytree=0.765327444916, learning_rate=

...

[CV]  colsample_bytree=0.883208373731, learning_rate=0.0801382547217, max_depth=10, n_estimators=1272, subsample=0.774922559607, total= 2.5min
[CV] colsample_bytree=0.883208373731, learning_rate=0.0801382547217, max_depth=10, n_estimators=1272, subsample=0.774922559607 
[CV]  colsample_bytree=0.883208373731, learning_rate=0.0801382547217, max_depth=10, n_estimators=1272, subsample=0.774922559607, total= 2.5min
[CV] colsample_bytree=0.763391059783, learning_rate=0.0917077332588, max_depth=8, n_estimators=1217, subsample=0.250856529716 
[CV]  colsample_bytree=0.763391059783, learning_rate=0.0917077332588, max_depth=8, n_estimators=1217, subsample=0.250856529716, total= 1.3min
[CV] colsample_bytree=0.763391059783, learning_rate=0.0917077332588, max_depth=8, n_estimators=1217, subsample=0.250856529716 
[CV]  colsample_bytree=0.763391059783, learning_rate=0.0917077332588, max_depth=8, n_estimators=1217, subsample=0.250856529716, total= 1.4min
[CV] colsample_bytree=0.763391059783, learning_r

[CV]  colsample_bytree=0.940090684067, learning_rate=0.0345623240299, max_depth=5, n_estimators=1183, subsample=0.406017239136, total= 1.1min
[CV] colsample_bytree=0.631187594724, learning_rate=0.0170475045762, max_depth=5, n_estimators=634, subsample=0.670643668177 
[CV]  colsample_bytree=0.631187594724, learning_rate=0.0170475045762, max_depth=5, n_estimators=634, subsample=0.670643668177, total=  38.8s
[CV] colsample_bytree=0.631187594724, learning_rate=0.0170475045762, max_depth=5, n_estimators=634, subsample=0.670643668177 
[CV]  colsample_bytree=0.631187594724, learning_rate=0.0170475045762, max_depth=5, n_estimators=634, subsample=0.670643668177, total=  31.5s
[CV] colsample_bytree=0.631187594724, learning_rate=0.0170475045762, max_depth=5, n_estimators=634, subsample=0.670643668177 
[CV]  colsample_bytree=0.631187594724, learning_rate=0.0170475045762, max_depth=5, n_estimators=634, subsample=0.670643668177, total=  32.2s
[CV] colsample_bytree=0.679791311028, learning_rate=0.064

[CV]  colsample_bytree=0.585818230682, learning_rate=0.0936435564334, max_depth=5, n_estimators=800, subsample=0.937721045305, total=  45.9s
[CV] colsample_bytree=0.585818230682, learning_rate=0.0936435564334, max_depth=5, n_estimators=800, subsample=0.937721045305 
[CV]  colsample_bytree=0.585818230682, learning_rate=0.0936435564334, max_depth=5, n_estimators=800, subsample=0.937721045305, total=  42.7s
[CV] colsample_bytree=0.585818230682, learning_rate=0.0936435564334, max_depth=5, n_estimators=800, subsample=0.937721045305 
[CV]  colsample_bytree=0.585818230682, learning_rate=0.0936435564334, max_depth=5, n_estimators=800, subsample=0.937721045305, total=  42.3s


[Parallel(n_jobs=1)]: Done 750 out of 750 | elapsed: 938.4min finished


CPU times: user 5d 3h 30min 8s, sys: 21min 45s, total: 5d 3h 51min 54s
Wall time: 15h 43min 14s


In [33]:
randomized_search.grid_scores_

/home/yuriyguts/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: -0.47500, std: 0.00043, params: {'colsample_bytree': 0.74267323180031286, 'learning_rate': 0.033076985073781255, 'max_depth': 9, 'n_estimators': 778, 'subsample': 0.29043055758760983},
 mean: -0.48568, std: 0.00053, params: {'colsample_bytree': 0.48308851626110311, 'learning_rate': 0.048381346629868781, 'max_depth': 5, 'n_estimators': 774, 'subsample': 0.34812924239791454},
 mean: -0.47722, std: 0.00034, params: {'colsample_bytree': 0.77879984040248496, 'learning_rate': 0.02613874089232876, 'max_depth': 8, 'n_estimators': 653, 'subsample': 0.66637526827723126},
 mean: -0.46532, std: 0.00059, params: {'colsample_bytree': 0.97239137419137323, 'learning_rate': 0.030531319747579694, 'max_depth': 11, 'n_estimators': 1034, 'subsample': 0.74313271520891644},
 mean: -0.48386, std: 0.00077, params: {'colsample_bytree': 0.24265369029979145, 'learning_rate': 0.031516681584218519, 'max_depth': 11, 'n_estimators': 828, 'subsample': 0.72564506619397329},
 mean: -0.48464, std: 0.00031, params:

In [22]:
randomized_search.best_params_

{'colsample_bytree': 0.47800317678119192,
 'learning_rate': 0.023478574372627188,
 'max_depth': 15,
 'n_estimators': 1399,
 'subsample': 0.90242029937509605}

In [ ]:
# skeval.plot.grid_search(
#     randomized_search.grid_scores_,
#     change='n_estimators',
#     kind='bar'
# )

In [23]:
# skeval.plot.grid_search(
#     search_result,
#     change=['colsample_bytree', 'subsample'],
#     subset={
#         'num_boost_round': 500,
#         'learning_rate': 0.05,
#         'max_depth': 10,
#     }
# )

## Evaluate Model

In [24]:
y_pred_train = model.predict(X_train)
y_pred_proba_train = model.predict_proba(X_train)[:, -1]

In [25]:
y_pred_val = model.predict(X_val)
y_pred_proba_val = model.predict_proba(X_val)[:, -1]

In [26]:
continuous_metrics = [log_loss, roc_auc_score]

In [27]:
binary_metrics = [accuracy_score, precision_score, recall_score]

### Train

In [28]:
for metric in continuous_metrics:
    print('{:20s}: {:10.5f}'.format(metric.__name__, metric(y_train, y_pred_proba_train)))
for metric in binary_metrics:
    print('{:20s}: {:10.5f}'.format(metric.__name__, metric(y_train, y_pred_train)))

log_loss            :    0.25620
roc_auc_score       :    0.99130
accuracy_score      :    0.91013
precision_score     :    0.80595
recall_score        :    0.99652


### Validation

In [29]:
for metric in continuous_metrics:
    print('{:20s}: {:10.5f}'.format(metric.__name__, metric(y_val, y_pred_proba_val)))
for metric in binary_metrics:
    print('{:20s}: {:10.5f}'.format(metric.__name__, metric(y_val, y_pred_val)))

log_loss            :    0.45364
roc_auc_score       :    0.85295
accuracy_score      :    0.75820
precision_score     :    0.63311
recall_score        :    0.82063


In [ ]:
raise ValueError('Stopping before the test set')

### Test

In [34]:
X_test = load(features_data_folder + 'X_test_summary_stats.pickle')

In [35]:
y_test = model.predict_proba(X_test)[:, -1]

In [36]:
submission_id = datetime.datetime.now().strftime('%Y-%m-%d-%H%M')

In [37]:
df_submission = pd.DataFrame({
    'test_id': range(len(y_test)),
    'is_duplicate': y_test
})

In [38]:
df_submission = df_submission[['test_id', 'is_duplicate']]

In [39]:
df_submission.head(10)

,test_id,is_duplicate
0,0,0.212164
1,1,0.636570
2,2,0.775008
3,3,0.000105
4,4,0.186864
5,5,0.005855
6,6,0.662158
7,7,0.601435
8,8,0.564478
9,9,0.254980


In [40]:
df_submission.to_csv(submissions_data_folder + submission_id + '-submission-draft.csv', header=True, index=None)